**Sinergia Dev** playbook base

- python install requirements

# Openai agents 

correr: openai-agents.py

In [ ]:
""" 
from agents import Agent, Runner
from pprint import pprint

oaAgent = Agent(name="Assistant", instructions="Eres un vendedor de pochoclos incansable y carismatico")
oaAgentResult = Runner.run_sync(oaAgent, "estoy por entrar al cine, alguna recomendacion para darme?")
pprint(oaAgentResult.final_output)
"""

![image.png](attachment/oa_agents.png)

# Agno Agents: Human in the loop example

#### ejemplo de Agent + custom tool

In [ ]:
## agno agents lib
# https://github.com/agno-agi/agno

# https://github.com/agno-agi/agno/tree/main/cookbook

In [ ]:
!pip install httpx rich agno

In [ ]:
"""🤝 Human-in-the-Loop: Adding User Confirmation to Tool Calls

This example shows how to implement human-in-the-loop functionality in your Agno tools.
It shows how to:
- Add pre-hooks to tools for user confirmation
- Handle user input during tool execution
- Gracefully cancel operations based on user choice

Some practical applications:
- Confirming sensitive operations before execution
- Reviewing API calls before they're made
- Validating data transformations
- Approving automated actions in critical systems

Run `pip install openai httpx rich agno` to install dependencies.
"""

import json
from textwrap import dedent
from typing import Iterator

import httpx
from agno.agent import Agent
from agno.exceptions import StopAgentRun
from agno.tools import FunctionCall, tool
from rich.console import Console
from rich.pretty import pprint
from rich.prompt import Prompt

# This is the console instance used by the print_response method
# We can use this to stop and restart the live display and ask for user confirmation
console = Console()


def pre_hook(fc: FunctionCall):
    # Get the live display instance from the console
    live = console._live

    # Stop the live display temporarily so we can ask for user confirmation
    live.stop()  # type: ignore

    # Ask for confirmation
    console.print(f"\nAbout to run [bold green]{fc.function.name}[/]")
    message = (
        Prompt.ask("Do you want to continue?", choices=["y", "n"], default="y")
        .strip()
        .lower()
    )

    # Restart the live display
    live.start()  # type: ignore

    # If the user does not want to continue, raise a StopExecution exception
    if message != "y":
        console.print(f"\n[bold red]Cancelled by user [/]")
        raise StopAgentRun(
            "Tool call cancelled by user",
            agent_message="Stopping execution as permission was not granted.",
        )


# USO DE CUSTOM TOOLS

@tool(pre_hook=pre_hook)
def get_top_hackernews_stories(num_stories: int) -> Iterator[str]:
    """Fetch top stories from Hacker News after user confirmation.

    Args:
        num_stories (int): Number of stories to retrieve

    Returns:
        str: JSON string containing story details
    """
    # Fetch top story IDs
    response = httpx.get("https://hacker-news.firebaseio.com/v0/topstories.json")
    story_ids = response.json()

    # Yield story details
    for story_id in story_ids[:num_stories]:
        story_response = httpx.get(
            f"https://hacker-news.firebaseio.com/v0/item/{story_id}.json"
        )
        story = story_response.json()
        if "text" in story:
            story.pop("text", None)
        yield json.dumps(story)


# Initialize the agent with a tech-savvy personality and clear instructions
agent = Agent(
    description="A Tech News Assistant that fetches and summarizes Hacker News stories",
    instructions=dedent("""\
        You are an enthusiastic Tech Reporter

        Your responsibilities:
        - Present Hacker News stories in an engaging and informative way
        - Provide clear summaries of the information you gather

        Style guide:
        - Use emoji to make your responses more engaging
        - Keep your summaries concise but informative
        - End with a friendly tech-themed sign-off\
    """),
    tools=[get_top_hackernews_stories],
    show_tool_calls=True,
    markdown=True,
)

# Example questions to try:
# - "What are the top 3 HN stories right now?"
# - "Show me the most recent story from Hacker News"
# - "Get the top 5 stories (you can try accepting and declining the confirmation)"
agent.print_response(
    "Show me a list of title from top 5 most recent story from Hacker News", stream=True, console=console
)

# View all messages
pprint(agent.run_response.messages)

INFO Setting default model to OpenAI Chat

About to run get_top_hackernews_stories

Do you want to continue? [y/n] (y):

[
│   Message(
│   │   role='system',
│   │   content='A Tech News Assistant that fetches and summarizes Hacker News stories\n<instructions>\nYou are an enthusiastic Tech Reporter\n\nYour responsibilities:\n- Present Hacker News stories in an engaging and informative way\n- Provide clear summaries of the information you gather\n\nStyle guide:\n- Use emoji to make your responses more engaging\n- Keep your summaries concise but informative\n- End with a friendly tech-themed sign-off    \n</instructions>\n\n<additional_information>\n- Use markdown to format your answers.\n</additional_information>',
│   │   name=None,
│   │   tool_call_id=None,
│   │   tool_calls=None,
│   │   audio=None,
│   │   images=None,
│   │   videos=None,
│   │   files=None,
│   │   audio_output=None,
│   │   image_output=None,
│   │   thinking=None,
│   │   redacted_thinking=None,
│   │   provider_data=None,
│   │   citations=None,
│   │   reasoning_content=None,
│   │   tool_name=None,
│   │   tool_args=None,
│   │   tool_call_error=None,
│   │   stop_after_tool_call=False,
│   │   add_to_agent_memory=True,
│   │   from_history=False,
│   │   metrics=MessageMetrics(
│   │   │   input_tokens=0,
│   │   │   output_tokens=0,
│   │   │   total_tokens=0,
│   │   │   audio_tokens=0,
│   │   │   input_audio_tokens=0,
│   │   │   output_audio_tokens=0,
│   │   │   cached_tokens=0,
│   │   │   reasoning_tokens=0,
│   │   │   prompt_tokens=0,
│   │   │   completion_tokens=0,
│   │   │   prompt_tokens_details=None,
│   │   │   completion_tokens_details=None,
│   │   │   additional_metrics=None,
│   │   │   time=None,
│   │   │   time_to_first_token=None,
│   │   │   timer=None
│   │   ),
│   │   references=None,
│   │   created_at=1745455352
│   ),
│   Message(
│   │   role='user',
│   │   content='Show me a list of title from top 5 most recent story from Hacker News',
│   │   name=None,
│   │   tool_call_id=None,
│   │   tool_calls=None,
│   │   audio=None,
│   │   images=None,
│   │   videos=None,
│   │   files=None,
│   │   audio_output=None,
│   │   image_output=None,
│   │   thinking=None,
│   │   redacted_thinking=None,
│   │   provider_data=None,
│   │   citations=None,
│   │   reasoning_content=None,
│   │   tool_name=None,
│   │   tool_args=None,
│   │   tool_call_error=None,
│   │   stop_after_tool_call=False,
│   │   add_to_agent_memory=True,
│   │   from_history=False,
│   │   metrics=MessageMetrics(
│   │   │   input_tokens=0,
│   │   │   output_tokens=0,
│   │   │   total_tokens=0,
│   │   │   audio_tokens=0,
│   │   │   input_audio_tokens=0,
│   │   │   output_audio_tokens=0,
│   │   │   cached_tokens=0,
│   │   │   reasoning_tokens=0,
│   │   │   prompt_tokens=0,
│   │   │   completion_tokens=0,
│   │   │   prompt_tokens_details=None,
│   │   │   completion_tokens_details=None,
│   │   │   additional_metrics=None,
│   │   │   time=None,
│   │   │   time_to_first_token=None,
│   │   │   timer=None
│   │   ),
│   │   references=None,
│   │   created_at=1745455352
│   ),
│   Message(
│   │   role='assistant',
│   │   content=None,
│   │   name=None,
│   │   tool_call_id=None,
│   │   tool_calls=[
│   │   │   {
│   │   │   │   'id': 'call_lY5FR5ivnbYdWVvMugp3kt58',
│   │   │   │   'type': 'function',
│   │   │   │   'function': {'name': 'get_top_hackernews_stories', 'arguments': '{"num_stories":5}'}
│   │   │   }
│   │   ],
│   │   audio=None,
│   │   images=None,
│   │   videos=None,
│   │   files=None,
│   │   audio_output=None,
│   │   image_output=None,
│   │   thinking=None,
│   │   redacted_thinking=None,
│   │   provider_data=None,
│   │   citations=None,
│   │   reasoning_content=None,
│   │   tool_name=None,
│   │   tool_args=None,
│   │   tool_call_error=None,
│   │   stop_after_tool_call=False,
│   │   add_to_agent_memory=True,
│   │   from_history=False,
│   │   metrics=MessageMetrics(
│   │   │   input_tokens=203,
│   │   │   output_tokens=22,
│   │   │   total_tokens=225,
│   │   │   audio_tokens=0,
│   │   │   input_audio_tokens=0,
│   │



---



# Memory with agno  

#### Ageno ofrece este tipo de memoria sqlite

In [2]:
from agno.agent import Agent
from agno.memory.v2.db.sqlite import SqliteMemoryDb
from agno.memory.v2.memory import Memory
from agno.models.openai import OpenAIChat
from agno.storage.sqlite import SqliteStorage
from rich.pretty import pprint

# UserId for the memories
user_id = "ava"
# Database file for memory and storage
db_file = "tmp/agent.db"

# Initialize memory.v2
memory = Memory(
    # Use any model for creating memories
    model=OpenAIChat(id="gpt-4.1"),
    db=SqliteMemoryDb(table_name="user_memories", db_file=db_file),
)
# Initialize storage
storage = SqliteStorage(table_name="agent_sessions", db_file=db_file)

# Initialize Agent
memory_agent = Agent(
    model=OpenAIChat(id="gpt-4.1"),
    # Store memories in a database
    memory=memory,
    # Give the Agent the ability to update memories
    enable_agentic_memory=True,
    # OR - Run the MemoryManager after each response
    enable_user_memories=True,
    # Store the chat history in the database
    storage=storage,
    # Add the chat history to the messages
    add_history_to_messages=True,
    # Number of history runs
    num_history_runs=3,
    markdown=True,
)

memory.clear()
memory_agent.print_response(
    "My name is Ava and I like to ski.",
    user_id=user_id,
    stream=True,
    stream_intermediate_steps=True,
)
print("Memories about Ava:")
pprint(memory.get_user_memories(user_id=user_id))

memory_agent.print_response(
    "I live in san francisco, where should i move within a 4 hour drive?",
    user_id=user_id,
    stream=True,
    stream_intermediate_steps=True,
)
print("Memories about Ava:")
pprint(memory.get_user_memories(user_id=user_id))

Memories about Ava:


[
│   UserMemory(
│   │   memory="User's name is Ava.",
│   │   topics=['name'],
│   │   input="Add a memory that the user's name is Ava and that Ava likes to ski.",
│   │   last_updated=datetime.datetime(2025, 4, 23, 21, 38, 3, 426525),
│   │   memory_id='58b1b69f-649f-44a0-beef-e25eb0c6de76'
│   ),
│   UserMemory(
│   │   memory='Ava likes to ski.',
│   │   topics=['hobbies', 'preferences'],
│   │   input="Add a memory that the user's name is Ava and that Ava likes to ski.",
│   │   last_updated=datetime.datetime(2025, 4, 23, 21, 38, 3, 430906),
│   │   memory_id='94002c91-1898-4514-a6fe-78e3b4e5dc5b'
│   )
]

Memories about Ava:


[
│   UserMemory(
│   │   memory='Ava lives in San Francisco.',
│   │   topics=['location', 'personal facts'],
│   │   input='I live in san francisco, where should i move within a 4 hour drive?',
│   │   last_updated=datetime.datetime(2025, 4, 23, 21, 38, 16, 146974),
│   │   memory_id='b4695510-672e-4e2a-8be4-78472e2fb0ec'
│   ),
│   UserMemory(
│   │   memory="User's name is Ava.",
│   │   topics=['name'],
│   │   input="Add a memory that the user's name is Ava and that Ava likes to ski.",
│   │   last_updated=datetime.datetime(2025, 4, 23, 21, 38, 3, 426525),
│   │   memory_id='58b1b69f-649f-44a0-beef-e25eb0c6de76'
│   ),
│   UserMemory(
│   │   memory='Ava likes to ski.',
│   │   topics=['hobbies', 'preferences'],
│   │   input="Add a memory that the user's name is Ava and that Ava likes to ski.",
│   │   last_updated=datetime.datetime(2025, 4, 23, 21, 38, 3, 430906),
│   │   memory_id='94002c91-1898-4514-a6fe-78e3b4e5dc5b'
│   )
]

# Memory w/mem0

#### este ejemplo usa mem0 para administrar la memoria del agente con busqueda semantica. 
- detalle: el ejemplo usa openai completions api

In [ ]:
# https://github.com/mem0ai/mem0

In [ ]:
!pip install mem0ai

In [7]:
from openai import OpenAI
from mem0 import Memory

openai_client = OpenAI()
memory = Memory()

def chat_with_memories(message: str, user_id: str = "default_user") -> str:
    # Retrieve relevant memories
    relevant_memories = memory.search(query=message, user_id=user_id, limit=3)
    memories_str = "\n".join(f"- {entry['memory']}" for entry in relevant_memories["results"])

    # Generate Assistant response
    system_prompt = f"You are a helpful AI. Answer the question based on query and memories.\nUser Memories:\n{memories_str}"
    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": message}]
    # Completion API
    response = openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages)
    assistant_response = response.choices[0].message.content

    # Create new memories from the conversation
    messages.append({"role": "assistant", "content": assistant_response})
    memory.add(messages, user_id=user_id)

    return assistant_response

def main():
    print("Chat with AI (type 'exit' to quit)")
    while True:
        user_input = input("You: ").strip()
        if user_input.lower() == 'exit':
            print("Goodbye!")
            break
        print(f"AI: {chat_with_memories(user_input)}")

if __name__ == "__main__":
    main()

Chat with AI (type 'exit' to quit)
AI: ¡Hola Matías! Es un placer conocerte. ¿En qué puedo ayudarte hoy?
AI: Te faltan 10 años para cumplir 40.
AI: Sé que te llamas Matías y que tienes 30 años porque lo compartiste anteriormente. ¿Hay algo más que te gustaría contarme o preguntar?


BadRequestError: Error code: 400 - {'error': {'message': "'$.input' is invalid. Please check the API reference: https://platform.openai.com/docs/api-reference.", 'type': 'invalid_request_error', 'param': None, 'code': None}}

# smolagents

#### este sistema para crear agentes se basa en la premisa de usar un coder agent para resolver como principal herramienta.

In [ ]:
!pip install smolagents ipywidgets
# https://github.com/huggingface/smolagents
# ejemplos: https://github.com/huggingface/smolagents/tree/main/examples

In [ ]:
# OBTENE TU HUGGING FACE TOKEN: tiene un free tier para usar deepseek y otros modelos hosteados.
# https://huggingface.co/settings/tokens

In [ ]:
from huggingface_hub import login
import getpass
import os


hf_api_key = os.environ.get("HF_API_TOKEN")
if hf_api_key is None:
    os.environ["HF_API_TOKEN"] = getpass.getpass("Enter HF API key: ")

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key is None:
    os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

login(hf_api_key)


In [ ]:
from smolagents import CodeAgent, DuckDuckGoSearchTool, InferenceClientModel

model = InferenceClientModel()
agent = CodeAgent(tools=[DuckDuckGoSearchTool()], model=model)

agent.run("How many seconds would it take for a leopard at full speed to run through Pont des Arts?")

╭──────────────────────────────────────────────────── New run ────────────────────────────────────────────────────╮
│                                                                                                                 │
│ How many seconds would it take for a leopard at full speed to run through Pont des Arts?                        │
│                                                                                                                 │
╰─ InferenceClientModel - Qwen/Qwen2.5-Coder-32B-Instruct ────────────────────────────────────────────────────────╯

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 1 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  pont_des_arts_length = web_search("length of Pont des Arts bridge")                                              
  print(pont_des_arts_length)                                                                                      
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
## Search Results

[Pont des Arts - Wikipedia](https://en.wikipedia.org/wiki/Pont_des_Arts)
Total length: 155 m (509 ft) Width: 11 m (36 ft) History; Construction start: 1981: Construction end: 1984: 
Statistics; Toll: Free both ways: Location; The Pont des Arts ... a nine-arch metallic bridge for pedestrians was 
constructed at the location of the present day Pont des Arts: this was the first metal bridge in Paris.

[Pont des Arts, Paris, France - LatLong](https://www.latlong.net/place/pont-des-arts-paris-france-32945.html)
Pont des Arts, Paris, France. Latitude and longitude coordinates are: 48.858334, 2.337369. Pont des Arts is a 
scenic pedestrian bridge over the Seine River in Paris, France. The bridge connects the Musée du Louvre and the 
Institut de France. It has a length of 155 m (509 ft) and a width of 11 m (36 ft). It was constructed between 1981 
and 1984.

[Love Lock bridge in Paris then & now: The Pont des 
Arts](https://cosmopoliclan.com/travel-with-kids/inspiration/love-lock-bridge-paris-pont-des-arts/)
The Pont des Arts or Passerelle des Arts is the pedestrian bridge over the Seine that connects the Louvre Museum on
the Right bank to the Institut de France on the Left Bank.As a result, it connects the 1st and the 6th Paris 
arrondissements. It is flanked by the Pont du Carrousel downstream and by the Pont-Neuf and the Île de la Cité, 
home to the Notre Dame cathedral, upstream.

[10 Picturesque Facts About The Pont Des Arts](https://art-facts.com/pont-des-arts-facts/)
1. It connects two famous buildings in the heart of Paris. The Pont des Arts literally translates to "The Bridge of
the Arts" and is also sometimes referred to as the "Passarelles des Arts."That's because it's a pedestrian-only 
bridge that allows people to easily cross the River Seine in the heart of Paris.

[Pont des Arts - The Love Bridge - Tour de 
France](https://www.tourdefrance-bridges.com/post/pont-des-arts-the-love-bridge)
The Pont des Arts was the first cast iron bridge in France and it paved the way for the increasing use of cast iron
in construction. ... The original bridge had 9 arches, was 155m in length and 11m wide and was made using cast 
iron. The number of arches was then reduced to 8 in 1952 due to the amalgamation of the two left-most arches.

[Pont des Arts Bridge - napoleon.org](https://www.napoleon.org/en/magazine/places/pont-des-arts-bridge/)
des Arts (Palace of the Arts), the bridge was christened the 'Pont de Arts'. As a simple footbridge crossing 
between the right and left banks of the Seine, the bridge was raised above the ground level of the two banks and 
comprised wooden planking resting on nine arches each of a span of 18.5 metres supported by masonry piles.

[Pont des Arts Paris: The romantic love bridge over the Seine](https://delveintoeurope.com/pont-des-arts-paris/)
The Pont des Arts bridge is one of the most famous Paris bridges, a beautiful ornate footbridge in a prime spot 
between the Louvre and Institut de France. The views from the Pont des Arts are incomparable, looking to either end
of the bridge, down the Seine or back upstream to the Pont Neuf, the Île de la Cité and towers of Notre Dame de ...

[History of the Pont des Arts in Paris - Come to 
Paris](https://www.cometoparis.com/paris-guide/paris-monuments/pont-des-arts-s959)
Unfortunately, in the 20th century, the aerial bombardments of two world wars, along with the damage caused by 
numerous boat collisions, left the Pont des Arts in an unsafe condition. A structural analysis in 1976 discovered 
various deficiencies, resulting in the bridge being closed to pedestrians the following year.

[The Pont des Arts Bridge - Tourism & Holiday Guide - France 
Voyage](https://www.france-voyage.com/tourism/pont-arts-bridge-2456.htm)
Located in the middle of Paris' historic city centre, between the prestigious Louvre Palace and the majestic French
Institute, the Arts Bridge (Pont des Arts) is surely one of Paris' most famous and romantic bridg

[Step 1: Duration 6.88 seconds| Input tokens: 2,056 | Output tokens: 89]

...

━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ Step 3 ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

─ Executing parsed code: ──────────────────────────────────────────────────────────────────────────────────────── 
  # Convert leopard's top speed from km/h to m/s                                                                   
  leopard_speed_mps = 58 * (1000 / 3600)                                                                           
                                                                                                                   
  # Length of the Pont des Arts bridge in meters                                                                   
  pont_des_arts_length_m = 155                                                                                     
                                                                                                                   
  # Calculate the time it would take for the leopard to run through the bridge                                     
  leopard_run_time = pont_des_arts_length_m / leopard_speed_mps                                                    
  print(f"The leopard would take {leopard_run_time:.2f} seconds to run through the Pont des Arts bridge.")         
  final_answer(leopard_run_time)                                                                                   
 ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────

Execution logs:
The leopard would take 9.62 seconds to run through the Pont des Arts bridge.

Out - Final answer: 9.620689655172415

[Step 3: Duration 12.29 seconds| Input tokens: 10,038 | Output tokens: 381]

9.620689655172415

# Browser-Use agent

#### este agente usa playwright para acceder a la web y extraer información

correr: browser-use-agent.py

In [ ]:
""" 
from langchain_openai import ChatOpenAI
from browser_use import Agent

import asyncio

llm = ChatOpenAI(model="gpt-4o")

async def main():
    agent = Agent(
        task="Compare the price of gpt-4o and DeepSeek-V3",
        llm=llm,
    )
    result = await agent.run()
    print(result)

asyncio.run(main())
"""

![image.png](attachment/browser_use_agent.png)